# Test Modules

In [2]:
# Set project root directory and add `src` to path
import sys
from pathlib import Path

PROJECT_ROOT = '/scratch/edk202/word2gm-fast'
project_root = Path(PROJECT_ROOT)
src_path = project_root / 'src'

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import the notebook setup utilities
from word2gm_fast.utils.notebook_setup import setup_testing_notebook, enable_autoreload, run_silent_subprocess

# Enable mixed precision for GPU training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

# Enable autoreload for development
enable_autoreload()

# Set up environment
env = setup_testing_notebook(project_root=PROJECT_ROOT)

# Extract commonly used modules for convenience
tf = env['tensorflow']
np = env['numpy']
pd = env['pandas']
print_resource_summary = env['print_resource_summary']

2025-06-27 15:53:18.486728: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-27 15:53:21.098363: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751054001.338911 1616803 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751054001.576679 1616803 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751054002.495703 1616803 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

<pre>Autoreload enabled</pre>

<pre>Project root: /scratch/edk202/word2gm-fast
TensorFlow version: 2.19.0
Device mode: GPU-enabled</pre>

<pre>Testing environment ready!</pre>

In [4]:
print_resource_summary()

<pre>SYSTEM RESOURCE SUMMARY
============================================================
Hostname: gr031.hpc.nyu.edu

Job Allocation:
   CPUs: 14
   Memory: 125.0 GB
   Requested partitions: rtx8000
   Actually running on: SSH failed: Host key verification failed.
   Job ID: 62896984
   Node list: gr031

GPU Information:
   CUDA GPUs detected: 1
   GPU 0: Quadro RTX 8000
      Memory: 0.5/45.0 GB (44.5 GB free)
      Temperature: 28°C
      Utilization: GPU 0%, Memory 0%

TensorFlow GPU Detection:
   TensorFlow detects 1 GPU(s)
      /physical_device:GPU:0, Memory growth: True
   Built with CUDA: True
============================================================</pre>

In [19]:
import subprocess

result = subprocess.run([
    'pytest',
    'tests/test_notebook_training.py'
], capture_output=True, text=True)

print(result.stdout)
if result.stderr:
    print(result.stderr)

============================= test session starts ==============================
platform linux -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0
rootdir: /scratch/edk202/word2gm-fast
plugins: anyio-4.9.0, timeout-2.4.0
collected 1 item

tests/test_notebook_training.py .                                        [100%]

=============================== warnings summary ===============================
tests/test_notebook_training.py::test_run_notebook_training_runs
tests/test_notebook_training.py::test_run_notebook_training_runs
tests/test_notebook_training.py::test_run_notebook_training_runs
  /ext3/miniforge3/envs/word2gm-fast2/lib/python3.12/site-packages/keras/src/backend/tensorflow/core.py:171: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
    return np.array(x)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
======================== 1 passed, 